# Use-case: Exploring perturbation effects

This tutorial shows how to perform the analysis we present in Figure 4 of the [pertpy preprint](https://www.biorxiv.org/content/10.1101/2024.08.04.606516v1).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pertpy as pt
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Load and preprocess the data

In [ ]:
adata = pt.dt.zhang_2021()
adata

In [ ]:
# Filter to tumor samples only
adata = adata[adata.obs["Origin"] == "t", :].copy()

# Filter out progression samples
adata = adata[adata.obs["Group"] != "Progression", :].copy()

# Subset to partial response and stable disease and rename PR and SD
adata = adata[adata.obs["Efficacy"].isin(["PR", "SD"]), :].copy()
adata.obs["Efficacy"] = adata.obs["Efficacy"].replace({"PR": "Partial response", "SD": "Stable disease"})

# Filter out Mix samples
adata = adata[adata.obs["Cluster"] != "Mix", :].copy()
adata

In [ ]:
adata.obs["Timepoint"] = adata.obs["Group"].copy()
adata.obs["Group"] = [f"{timepoint.split('-')[0]}-treat., {response}" for timepoint, response in zip(adata.obs["Timepoint"], adata.obs["Efficacy"])]
adata.obs["Group"].value_counts()

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor="seurat_v3")

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
adata

In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="Treatment")

In [ ]:
sc.pl.umap(adata, color="Group")

In [ ]:
cell_type_converter = {
    "t_Bfoc": "B cell",
    "t_Bmem": "B cell",
    "t_Bn": "B cell",
    "t_CD4": "CD4 T cell",
    "t_CD8": "CD8 T cell",
    "t_ILC": "ILC cell",
    "t_Tact": "Activated T cell",
    "t_Tn": "Naive T cell",
    "t_Tprf": "Proliferating T cell",
    "t_cDC": "Dendritic cell",
    "t_mDC": "Dendritic cell",
    "t_macro": "Macrophage",
    "t_mast": "Mast cell",
    "t_mono": "Monocyte",
    "t_pB": "B cell",
    "t_pDC": "Dendritic cell",
}

cell_types = []
for ct in adata.obs["Cluster"]:
    for key, value in cell_type_converter.items():
        if ct.startswith(key):
            cell_types.append(value)
            break
    else:
        cell_types.append("n.a.")
adata.obs["Celltype"] = cell_types

sc.pl.umap(adata, color="Celltype")

## Using a distance metric to rank perturbation effects

In [ ]:
def filter_data(adata_temp):
    isecs = pd.crosstab(adata_temp.obs["Cluster"], adata_temp.obs["Group"])
    celltypes = isecs[(isecs >0).all(axis=1)].index.values.tolist()
    adata_temp = adata_temp[adata_temp.obs["Cluster"].isin(celltypes)]
    return adata_temp

In [ ]:
adata_chemo = adata[adata.obs["Treatment"] == "Chemo"]
adata_chemo = filter_data(adata_chemo)
adata_chemo.obs["Group"].value_counts()

In [ ]:
adata_chemo_pdl1 = adata[adata.obs["Treatment"] == "Anti-PD-L1+Chemo"]
adata_chemo_pdl1 = filter_data(adata_chemo_pdl1)
adata_chemo_pdl1.obs["Group"].value_counts()

In [ ]:
distance = pt.tl.Distance("mse", obsm_key="X_pca")
df_all = distance.pairwise(adata, groupby="Group", show_progressbar=False)

df_chemo = distance.pairwise(adata_chemo, groupby="Group", show_progressbar=False)

df_chemo_pdl1 = distance.pairwise(adata_chemo_pdl1, groupby="Group", show_progressbar=False)

In [ ]:
global_max = max(df_all.max(axis=None), df_chemo.max(axis=None), df_chemo_pdl1.max(axis=None))
global_min = min(df_all.min(axis=None), df_chemo.min(axis=None), df_chemo_pdl1.min(axis=None))

order = df_all.index.values

In [ ]:
_, ax = plt.subplots(figsize=(4, 3.5))
sns.heatmap(df_all, annot=True, fmt=".2f", vmin=global_min, vmax=global_max, cmap="Reds", ax=ax)
plt.show()

In [ ]:
df_chemo = df_chemo.loc[order, order]
_, ax = plt.subplots(figsize=(4, 3.5))
sns.heatmap(df_chemo, annot=True, fmt=".2f", vmin=global_min, vmax=global_max, cmap="Reds", ax=ax)
plt.show()

In [ ]:
df_chemo_pdl1 = df_chemo_pdl1.loc[order, order]
_, ax = plt.subplots(figsize=(4, 3.5))
sns.heatmap(df_chemo_pdl1, annot=True, fmt=".2f", vmin=global_min, vmax=global_max, cmap="Reds", ax=ax)
plt.show()

## Identifying changes in cell type composition

In [ ]:
# Get reference cell type
sccoda_model = pt.tl.Sccoda()

sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="Cluster",
    sample_identifier="Sample",
    covariate_obs=["Group"],
)

sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula=f"Group",
    reference_cell_type="automatic",
    automatic_reference_absence_threshold=0.1,
)